# Assignment 1 - Building a Vision Model with Keras

In this assignment, you will build a simple vision model using Keras. The goal is to classify images from the Fashion MNIST dataset, which contains images of clothing items.

You will:
1. Load and inspect the Fashion MNIST dataset.
2. Run a simple baseline model to establish a performance benchmark.
3. Build and evaluate a simple CNN model, choosing appropriate loss and metrics.
4. Design and run controlled experiments on one hyperparameter (e.g., number of filters, kernel size, etc.) and one regularization technique (e.g., dropout, L2 regularization).
5. Analyze the results and visualize the model's performance.

# 1. Loading and Inspecting the Dataset

Fashion MNIST is a dataset of grayscale images of clothing items, with 10 classes. Each image is 28x28 pixels, like the MNIST dataset of handwritten digits. Keras provides a convenient way to load this dataset. 

In this section, you should:

- [ ] Inspect the shapes of the training and test sets to confirm their size and structure.
- [ ] Convert the labels to one-hot encoded format if necessary. (There is a utility function in Keras for this.)
- [ ] Visualize a few images from the dataset to understand what the data looks like.

In [2]:
from tensorflow.keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize the pixel values to be between 0 and 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Classes in the Fashion MNIST dataset
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [3]:
# Inspect the shapes of the datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# Convert labels to one-hot encoding
from tensorflow.keras.utils import to_categorical

y_train_cat = to_categorical(y_train, num_classes=10)
y_test_cat = to_categorical(y_test, num_classes=10)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [4]:
import matplotlib.pyplot as plt
# Verify the data looks as expected

for i in range(5):
    plt.subplot(1, 5, i + 1)  # 1 row, 5 columns
    plt.imshow(X_train[i], cmap='gray')
    plt.title(class_names[y_train[i]])
    
plt.show()

: 

Reflection: Does the data look as expected? How is the quality of the images? Are there any issues with the dataset that you notice?

**Your answer here**
# The data looks good - some confusion between ankle boots vs sandals
# quality of image is decent (not great)


# 2. Baseline Model

In this section, you will create a linear regression model as a baseline. This model will not use any convolutional layers, but it will help you understand the performance of a simple model on this dataset.
You should:
- [ ] Create a simple linear regression model using Keras.
- [ ] Compile the model with an appropriate loss function and optimizer.
- [ ] Train the model on the training set and evaluate it on the test set.

A linear regression model can be created using the `Sequential` API in Keras. Using a single `Dense` layer with no activation function is equivalent to a simple linear regression model. Make sure that the number of units in the output layer matches the number of classes in the dataset.

Note that for this step, we will need to use `Flatten` to convert the 2D images into 1D vectors before passing them to the model. Put a `Flatten()` layer as the first layer in your model so that the 2D image data can be flattened into 1D vectors.

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize the pixel values to be between 0 and 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Classes in the Fashion MNIST dataset
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

# Create a simple linear regression model
model = Sequential()
# You can use `model.add(<layer>)` to add layers to the model
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(10))

# Compile the model using `model.compile()`
model.compile(optimizer=Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with `model.fit()`
model1 = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model with `model.evaluate()`
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"accuracy: {test_accuracy:.3f}")

Epoch 1/10
1688/1688 [==============================] - 4s 2ms/step - loss: 2.3555 - accuracy: 0.2842 - val_loss: 2.3014 - val_accuracy: 0.1888
Epoch 2/10
1688/1688 [==============================] - 3s 2ms/step - loss: 2.3028 - accuracy: 0.1739 - val_loss: 2.3022 - val_accuracy: 0.1817
Epoch 3/10
1688/1688 [==============================] - 3s 2ms/step - loss: 2.3031 - accuracy: 0.1654 - val_loss: 2.3026 - val_accuracy: 0.1698
Epoch 4/10
1688/1688 [==============================] - 4s 2ms/step - loss: 2.3031 - accuracy: 0.1620 - val_loss: 2.3026 - val_accuracy: 0.1872
Epoch 5/10
1688/1688 [==============================] - 3s 2ms/step - loss: 2.3027 - accuracy: 0.1820 - val_loss: 2.3026 - val_accuracy: 0.1938
Epoch 6/10
1688/1688 [==============================] - 3s 2ms/step - loss: 2.3027 - accuracy: 0.1925 - val_loss: 2.3026 - val_accuracy: 0.2197
Epoch 7/10
1688/1688 [==============================] - 4s 2ms/step - loss: 2.3025 - accuracy: 0.2072 - val_loss: 2.3026 - val_accuracy:

Reflection: What is the performance of the baseline model? How does it compare to what you expected? Why do you think the performance is at this level?

**Your answer here**

In [ ]:
# The performance of the model is poor and not learning anything as it iterates. We aren't using an 
# activiation and the images are complex and will need to use ReLu for complex images. 
# model overall is underfitting


# 3. Building and Evaluating a Simple CNN Model

In this section, you will build a simple Convolutional Neural Network (CNN) model using Keras. A convolutional neural network is a type of deep learning model that is particularly effective for image classification tasks. Unlike the basic neural networks we have built in the labs, CNNs can accept images as input without needing to flatten them into vectors.

You should:
- [ ] Build a simple CNN model with at least one convolutional layer (to learn spatial hierarchies in images) and one fully connected layer (to make predictions).
- [ ] Compile the model with an appropriate loss function and metrics for a multi-class classification problem.
- [ ] Train the model on the training set and evaluate it on the test set.

Convolutional layers are designed to accept inputs with three dimensions: height, width and channels (e.g., RGB for color images). For grayscale images like those in Fashion MNIST, the input shape will be (28, 28, 1).

When you progress from the convolutional layers to the fully connected layers, you will need to flatten the output of the convolutional layers. This can be done using the `Flatten` layer in Keras, which doesn't require any parameters.

In [9]:
from keras.layers import Conv2D

from tensorflow.keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize the pixel values to be between 0 and 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Reshape the data to include the channel dimension
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# Create a simple CNN model
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model1 = model.fit(X_train, y_train, epochs=3, batch_size=64, validation_split=0.1)

# Evaluate the model

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"accuracy: {test_accuracy:.3f}")

Epoch 1/3
844/844 [==============================] - 7s 8ms/step - loss: 0.4713 - accuracy: 0.8367 - val_loss: 0.3750 - val_accuracy: 0.8670
Epoch 2/3
844/844 [==============================] - 7s 8ms/step - loss: 0.3241 - accuracy: 0.8856 - val_loss: 0.3116 - val_accuracy: 0.8900
Epoch 3/3
313/313 [==============================] - 8s 26ms/step - loss: 0.3131 - accuracy: 0.8884
accuracy: 0.888


Reflection: Did the CNN model perform better than the baseline model? If so, by how much? What do you think contributed to this improvement?

**Your answer here**

In [ ]:
# Significant improvement in performance almost 70% highe accuracy 
# Using convolutional layers it was able to extract the features better and CNN architecture is better
# for image classification tasks

# 3. Designing and Running Controlled Experiments

In this section, you will design and run controlled experiments to improve the model's performance. You will focus on one hyperparameter and one regularization technique.
You should:
- [ ] Choose one hyperparameter to experiment with (e.g., number of filters, kernel size, number of layers, etc.) and one regularization technique (e.g., dropout, L2 regularization). For your hyperparameter, you should choose at least three different values to test (but there is no upper limit). For your regularization technique, simply test the presence or absence of the technique.
- [ ] Run experiments by modifying the model architecture or hyperparameters, and evaluate the performance of each model on the test set.
- [ ] Record the results of your experiments, including the test accuracy and any other relevant metrics.
- [ ] Visualize the results of your experiments using plots or tables to compare the performance of different models.

The best way to run your experiments is to create a `for` loop that iterates over a range of values for the hyperparameter you are testing. For example, if you are testing different numbers of filters, you can create a loop that runs the model with 32, 64, and 128 filters. Within the loop, you can compile and train the model, then evaluate it on the test set. After each iteration, you can store the results in a list or a dictionary for later analysis.

Note: It's critical that you re-initialize the model (by creating a new instance of the model) before each experiment. If you don't, the model will retain the weights from the previous experiment, which can lead to misleading results.

In [11]:
# A. Test Hyperparameters

from keras.layers import Conv2D
from tensorflow.keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize the pixel values to be between 0 and 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Reshape the data to include the channel dimension
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

filter_values = [32, 64, 128]

for i in filter_values:

    # Create a simple CNN model
    model = Sequential()
    model.add(Conv2D(i, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    # Train the model
    model1 = model.fit(X_train, y_train, epochs=3, batch_size=64, validation_split=0.1)

    # Evaluate the model

    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f"accuracy: {test_accuracy:.3f}")





Epoch 1/3
844/844 [==============================] - 10s 11ms/step - loss: 0.4274 - accuracy: 0.8513 - val_loss: 0.3212 - val_accuracy: 0.8843
Epoch 2/3
844/844 [==============================] - 10s 12ms/step - loss: 0.2904 - accuracy: 0.8961 - val_loss: 0.3116 - val_accuracy: 0.8852
Epoch 3/3
313/313 [==============================] - 10s 33ms/step - loss: 0.3058 - accuracy: 0.8900
accuracy: 0.890
Epoch 1/3
844/844 [==============================] - 16s 18ms/step - loss: 0.4146 - accuracy: 0.8540 - val_loss: 0.3248 - val_accuracy: 0.8828
Epoch 2/3
844/844 [==============================] - 14s 17ms/step - loss: 0.2870 - accuracy: 0.8975 - val_loss: 0.2933 - val_accuracy: 0.8970
Epoch 3/3
313/313 [==============================] - 11s 34ms/step - loss: 0.3098 - accuracy: 0.8921
accuracy: 0.892
Epoch 1/3
844/844 [==============================] - 27s 32ms/step - loss: 0.4032 - accuracy: 0.8583 - val_loss: 0.3286 - val_accuracy: 0.8847
Epoch 2/3
844/844 [==============================] 

In [12]:
# B. Test presence or absence of regularization

from keras.layers import Dropout

from tensorflow.keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize the pixel values to be between 0 and 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Reshape the data to include the channel dimension
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

dropout_options = [False, True]

for i in dropout_options:
    
    # Create a simple CNN model
    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1))) # set to 64 since best performance
    if i: # add drop out parameter
        model.add(Dropout(0.5))  
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    # Train the model
    model1 = model.fit(X_train, y_train, epochs=3, batch_size=64, validation_split=0.1)

    # Evaluate the model

    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f"accuracy: {test_accuracy:.3f}")

Epoch 1/3
844/844 [==============================] - 16s 18ms/step - loss: 0.4134 - accuracy: 0.8540 - val_loss: 0.3267 - val_accuracy: 0.8858
Epoch 2/3
844/844 [==============================] - 15s 18ms/step - loss: 0.2875 - accuracy: 0.8977 - val_loss: 0.2929 - val_accuracy: 0.8935
Epoch 3/3
313/313 [==============================] - 11s 36ms/step - loss: 0.3030 - accuracy: 0.8917
accuracy: 0.892
Epoch 1/3
844/844 [==============================] - 33s 39ms/step - loss: 0.4241 - accuracy: 0.8503 - val_loss: 0.3176 - val_accuracy: 0.8847
Epoch 2/3
844/844 [==============================] - 31s 36ms/step - loss: 0.3051 - accuracy: 0.8917 - val_loss: 0.2986 - val_accuracy: 0.8937
Epoch 3/3
313/313 [==============================] - 11s 36ms/step - loss: 0.3107 - accuracy: 0.8894
accuracy: 0.889


Reflection: Report on the performance of the models you tested. Did any of the changes you made improve the model's performance? If so, which ones? What do you think contributed to these improvements? Finally, what combination of hyperparameters and regularization techniques yielded the best performance?

**Your answer here**

In [ ]:
# Changing the filter and running with no dropout yielded the best results. 
# filtering above 64 didn't improve results but increased time. 
# dropout didn't provide improvement -> could be because my epoch was set to 3...
# longer training would help and make dropout useful

# 5. Training Final Model and Evaluation

In this section, you will train the final model using the best hyperparameters and regularization techniques you found in the previous section. You should:
- [ ] Compile the final model with the best hyperparameters and regularization techniques.
- [ ] Train the final model on the training set and evaluate it on the test set.
- [ ] Report the final model's performance on the test set, including accuracy and any other relevant metrics.

In [14]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize the pixel values to be between 0 and 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Reshape the data to include the channel dimension
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# Build model with filter set to 64 and will utilize dropout with larger epoch
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    Dropout(0.5),
    Flatten(),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model1 = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

# Evaluate on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Final model test accuracy: {test_accuracy:.3f}")

Epoch 1/10
844/844 [==============================] - 42s 49ms/step - loss: 0.4288 - accuracy: 0.8491 - val_loss: 0.3488 - val_accuracy: 0.8738
Epoch 2/10
844/844 [==============================] - 31s 37ms/step - loss: 0.3106 - accuracy: 0.8889 - val_loss: 0.3085 - val_accuracy: 0.8913
Epoch 3/10
844/844 [==============================] - 30s 36ms/step - loss: 0.2782 - accuracy: 0.9008 - val_loss: 0.2977 - val_accuracy: 0.8940
Epoch 4/10
844/844 [==============================] - 30s 36ms/step - loss: 0.2577 - accuracy: 0.9068 - val_loss: 0.2901 - val_accuracy: 0.8977
Epoch 5/10
844/844 [==============================] - 31s 36ms/step - loss: 0.2410 - accuracy: 0.9127 - val_loss: 0.2943 - val_accuracy: 0.8993
Epoch 6/10
844/844 [==============================] - 31s 36ms/step - loss: 0.2288 - accuracy: 0.9168 - val_loss: 0.2783 - val_accuracy: 0.9042
Epoch 7/10
844/844 [==============================] - 31s 36ms/step - loss: 0.2169 - accuracy: 0.9208 - val_loss: 0.2754 - val_accuracy:

Reflection: How does the final model's performance compare to the baseline and the CNN model? What do you think contributed to the final model's performance? If you had time, what other experiments would you run to further improve the model's performance?

**Your answer here**

In [15]:
# The final model is superior achieving an accuracy of 90%
# I think the dropouthelped with generalization 
# If I had more time I would try to tune the dropout and increase training epoch
# maybe batch normalization to boost performance


🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.
### Submission Parameters:
* Submission Due Date: `23:59 PM - 06/07/2025`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb)
    * The Lab 1 notebook (labs/lab_1.ipynb)
    * The Lab 2 notebook (labs/lab_2.ipynb)
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/deep_learning/pull/<pr_id>`
* Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.
Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.
If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-6-ml-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.